In [7]:
import json
import numpy as np
from socket import socket
from a2gmeasurements import HelperA2GMeasurements, GimbalRS2, GpsSignaling
import time, timeit

In [8]:
meas_number = '1_AGAIN_2_40'

OPERATE GROUND GIMBAL WITH CMDS

In [9]:
myGimbal = GimbalRS2()
myGimbal.start_thread_gimbal()

uptime library not available, timestamps are relative to boot time and not to Epoch UTC
Traceback (most recent call last):
  File "c:\Users\manifold-uav-vtt\anaconda3\envs\uavCom\lib\site-packages\can\interfaces\pcan\pcan.py", line 71, in <module>
    import uptime
ModuleNotFoundError: No module named 'uptime'


The driver is not loaded

[DEBUG]: Gimbal thread NOT started


In [10]:
myGimbal.setPosControl(yaw=1650, roll=0, pitch=131)

AttributeError: 'GimbalRS2' object has no attribute 'actual_bus'

In [ ]:
myGimbal.stop_thread_gimbal()
time.sleep(0.05)
myGimbal.actual_bus.shutdown()

OPEN MEASUREMENT HELPER CLASS

In [ ]:
host = ''
myHelper = HelperA2GMeasurements('GROUND', host, DBG_LVL_0=False, 
                                 DBG_LVL_1=True, 
                                 IsGPS=True, 
                                 IsGimbal=True, 
                                 IsSignalGenerator=True,
                                 F0=28.95,
                                 L0=20)

PEAK SIGNAL GENERATOR CALIBRATION

In [ ]:
# Turn On RF output
myHelper.inst.write('RF1\n')
time.sleep(2)
# Turn Off RF output
myHelper.inst.write('RF0\n')

to_save_now = json.dumps(myHelper.mySeptentrioGPS.NMEA_buffer[-1])
filename = 'MEASUREMENT_' + meas_number + '_SIGNAL_GENERATOR_CALIBRATION_PEAK'
f = open(filename + '.json', 'w')
f.write(to_save_now)
f.close()

ROTATION GROUND GIMBAL MEASUREMENT

In [ ]:
# 9. Request Gimbal orientation
myHelper.myGimbal.request_current_position()
time.sleep(0.002)

# Gimbal assumed to be not moving
yaw_now = myHelper.myGimbal.yaw
#roll_now = myHelper.myGimbal.roll

# 10. Move GROUND gimbal in 30 deg steps
N_ang_steps = 360/30
reset_yaw_gimbal = myHelper.az_rot_gnd_gimbal_toggle_sig_generator(int(yaw_now*10), 
                                                  131, 
                                                  int(N_ang_steps), 
                                                  meas_number=meas_number)

# 11. Make the gimbal move in the opposite direction to unfold cables

print('ANGLES TO UNFOLD CABLE IN GIMBAL: ', reset_yaw_gimbal)
for i in reset_yaw_gimbal:
    myHelper.myGimbal.setPosControl(yaw=i, pitch=131, roll=0)
    time.sleep(1)

myHelper.HelperA2GStopCom()
#input('Stop test?')

POWER SPECTRUM MEASUREMENT CHECK

In [ ]:
from datetime import datetime
data = []
n_samp = []
meas_timestamp = []
def iterate_until(iterable):
    
    """Returns an iterator that returns all keys or values
       of a (nested) iterable.
       
       Arguments:
           - iterable: <list> or <dictionary>
           - returned: <string> "key" or "value"
           
       Returns:
           - <iterator>
    """
    if isinstance(iterable, dict):
        for key, value in iterable.items():
            if key == 'TimeStamp':
                    if value['@year']:
                        meas_date = datetime(year=int(value['@year']), 
                                             month=int(value['@month']),
                                             day=int(value['@day']),
                                             hour=int(value['@hour']),
                                             minute=int(value['@minute']),
                                             second=int(value['@second']))
                        meas_timestamp.append(datetime.timestamp(meas_date))
            if not (isinstance(value, dict) or isinstance(value, list)):
                if key == '@id':
                    try:
                        n_samp.append(int(value))
                        data.append(iterable['@value'])
                    except:
                        1
            else:
                iterate_until(value)

    elif isinstance(iterable, list):
        #print(iterable)
        #data.append(iterable)
        for i in iterable:
            iterate_until(i)
      
    return data, n_samp

In [ ]:
from cgi import test
import xmltodict
import json
import pprint
import os
import matplotlib.pyplot as plt
import numpy as np

root_directory = "C://Users//jvjulian//OneDrive - Teknologian Tutkimuskeskus VTT//Documents//Measurement 1//"
test_folders = "20221012"

# Each folder has 498 files (except first file of first folder)
# Each file has around 650 points
# These are consecutive points

max_num_points_per_plot = 50000
cnt_exceptions = 0
for i in os.listdir(root_directory):
    if test_folders in i:
        this_folder = root_directory + i
        for j in os.listdir(this_folder):
            filename = this_folder + '//' + j

            with open(filename, 'r') as fd:
                try:
                    doc = xmltodict.parse(fd.read(), process_namespaces=True)
                except:
                    cnt_exceptions = cnt_exceptions + 1
                    print('EXCEPTION on file ', filename)
            data, n_samp = iterate_until(doc)

y = [np.nan if i=='nan' else float(i) for i in data]

In [ ]:
from serial.tools.list_ports import comports

for (this_port, desc, extra_info) in sorted(comports()):
    print(desc)
    



Intel(R) Active Management Technology - SOL (COM3)
Septentrio Virtual USB COM Port 1 (COM11)
Septentrio Virtual USB COM Port 2 (COM12)


In [1]:
from a2gmeasurements import GimbalGremsyH16, SBUSEncoder

gh16 = GimbalGremsyH16()

[DEBUG]: POSITIVE SPEEDS (LEFT), AZIMUTH, R^2 Score Linear Reg:  0.9828922068260325
[DEBUG]: NEGATIVE SPEEDS (RIGHT), AZIMUTH, R^2 Score Linear Reg:  0.9708341771256139
[DEBUG]: NEGATIVE SPEEDS (DOWN), ELEVATION, R^2 Score Linear Reg:  0.812453157362137
[DEBUG]: POSITIVE SPEEDS (UP), ELEVATION, R^2 Score Linear Reg:  0.812453157362137
[DEBUG]: Connected to IMU
[DEBUG]: READ IMU THREAD STARTED


In [2]:
gh16.start_conn()


[DEBUG_0]: serial port connected

[DEBUG]: SBUS threading started
[DEBUG]: Defining HOME for Gremsy... This might take a second
TIME SPENT DEFINE HOME POSITION: 0.6173861026763916


In [3]:
gh16.home_position

{'YAW': 45.875, 'PITCH': 2.625}

In [6]:
gh16.sbus.move_gimbal(0, 10, 3)

In [7]:
gh16.stop_conn()

Exception in thread Thread-5 (receive_imu_data):
Traceback (most recent call last):
  File "c:\Users\manifold-uav-vtt\anaconda3\envs\uavCom\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Users\manifold-uav-vtt\anaconda3\envs\uavCom\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\manifold-uav-vtt\Documents\a2gMeasurements\a2gmeasurements.py", line 2362, in receive_imu_data
    self.last_imu_reading = {'YAW': float(data[0]), 'PITCH': float(data[1]), 'ROLL': float(data[2])}
ValueError: could not convert string to float: ''
